# 1) Creacion de dataset entrenamiento

### Librerias

In [2]:
import glob
import os
from tqdm import tqdm


### Instalacion de pysatproc (correr solo una vez)

In [3]:
#!pip instal pysatproc

In [31]:
#indico path a imagenes y anotaciones de entrenamiento y donde guardo el dataset. Tambien, size y step size.
path_to_files = '../images-S2/zona_3/2021-01-01_2021-03-01/*.tif'             # Path to the satellite images folder  ANTES:'data/img_train/zona_2/*.tif'  
output_folder = "../dataset/data_train/zona_3/v2/80_40/"        # Destination path <ndim>/<size>_<step-size>/

vector_file_path = "../data/shp/gt/R3/Asentamiento_R3_v2.geojson"  # Path to vectorial file
vector_aoi_file_path = vector_file_path

size = 80            # size (pixels) of each dataset image generated
step_size = 40         # step  size of the sliding window 

In [32]:
#satproc_extract_chips --help

In [33]:
#Creo el dataset para entrenamiento (imag + anotaciones)
!satproc_extract_chips \
    $path_to_files \
    -o  $output_folder \
    --size $size \
    --step-size $step_size \
    --aoi $vector_file_path \
    --labels $vector_file_path \
    --label-property 'class' \
    --classes 'A' \
    --rescale \
    --rescale-mode s2_rgb_extra --lower-cut 1 --upper-cut 100


 62%|████████████████████████████▏                | 5/8 [00:42<00:22,  7.48s/it]
0it [00:00, ?it/s]
 88%|███████████████████████████████████████▍     | 7/8 [00:48<00:04,  4.91s/it]
0it [00:00, ?it/s]
100%|█████████████████████████████████████████████| 8/8 [00:49<00:00,  6.23s/it]


# 2) Entrenamiento

In [34]:
#!pip install unetseg

In [35]:
#importo la red neuronal
from unetseg.train import TrainConfig, train
from unetseg.evaluate import plot_data_generator
import os

In [42]:
#especifico los parametros. Determino donde guardar el modelo
config = TrainConfig(width=80,  #>80 , multiplo de 16 , <320 (depende)
                     height=80, 
                     n_channels=4, 
                     n_classes=1,
                     apply_image_augmentation=True,
                     seed=42,
                     epochs=20, 
                     batch_size=16, 
                     steps_per_epoch=80, ## > #img/batch_size 10%
                     early_stopping_patience=5,
                     validation_split=0.2, 
                     test_split=0.1,  
                     model_architecture='unet', #unetplusplus
                     images_path=os.path.join("../dataset/data_train","zona_3","v2","80_40"), 
                     model_path=os.path.join('../data/weights/model/zona3/', 'UNet_techo_4D_spe100_img80x80_size80_sz40.h5'),
                     evaluate=True ,
                     class_weights = [1]) 

In [43]:
#Corro el modelo con los parametros establecidos
res_config = train(config)

ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concat axis. Got inputs shapes: [(None, 4, 4, 512), (None, 5, 5, 512)]

In [ ]:
plot_data_generator(num_samples=3, fig_size=(10, 10), train_config = config,img_ch = 3)